In [1]:
import polars as pl
import os

pl.Config.set_fmt_str_lengths(5000)
pl.Config.set_tbl_width_chars(5000)
pl.Config.set_tbl_cols(25)
pl.Config.set_tbl_rows(1000)

DATASET_PATH = os.path.join(os.path.abspath("."), "datasets/911/911_metadata.csv")
AGG_CAT = os.path.join(os.path.abspath("."), "datasets/911/agg_cat.json")
AGG_CLUST = os.path.join(os.path.abspath("."), "datasets/911/agg_clust.json")

In [2]:
%time
df = pl.read_csv(DATASET_PATH)

head = df.head()
description = df.describe()

CPU times: user 2 μs, sys: 0 ns, total: 2 μs
Wall time: 3.58 μs


In [3]:
print(head)
print(description)

shape: (5, 13)
┌─────┬──────────┬─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬──────────────────────────────────┬──────┬──────────┬────────┬─────────────────┬─────────────┬──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬───────────────┬───────┬─────────────────────────────┐
│ id  ┆ event_id ┆ link                                                                                                                ┆ title                            ┆ date ┆ state    ┆ deaths ┆ potential_death ┆ false_alarm ┆ description                                                                                                                                       

In [4]:
%time
columns = df.columns
df_len = len(df)
unique_titles = df["title"].unique()

CPU times: user 2 μs, sys: 0 ns, total: 2 μs
Wall time: 3.81 μs


In [5]:
print("Columns:\n", columns)
print("Len: ", df_len)
print("Titles:\n", unique_titles, len(unique_titles))

Columns:
 ['id', 'event_id', 'link', 'title', 'date', 'state', 'deaths', 'potential_death', 'false_alarm', 'description', 'deaths_binary', 'break', 'filename']
Len:  710
Titles:
 shape: (467,)
Series: 'title' [str]
[
	"Murder suicide"
	"Murder victim 911 – Fla."
	"School lock-down"
	"Baby Delivery 911 – Tex."
	"Assault 911 call"
	"Dog attack on 911"
	"Police shooting"
	"Air Force One Fly-Over"
	"Cab death"
	"Worker finds skull"
	"TJ Maxx robbery"
	"Murder-suicide – Wisc."
	"Heatstroke death"
	"Murder-Suicide"
	"Shoot-out"
	"Religious retreat shootings"
	"Med patient dies"
	"Home invasion murders"
	"Unconscious woman"
	"Small plane crashes"
	"911 call from McDonald’s shooting"
	"Murder calls"
	"Daytime burglary"
	"Pursuit"
	"Grandmother rescue – Mi."
	"Hostages taken"
	"Infant death"
	"Face attack"
	"Accusation Against Officer"
	"Man With Gun"
	"Murder"
	"Baby not breathing"
	"Motorist shot"
	"Murder 911 Call – SC"
	"Drowning in car"
	"Plane crash"
	"Mass murders"
	"Burglary In-Progress

In [6]:
%time
from sklearn.cluster import DBSCAN

proc_df = df.filter(df["false_alarm"] == 0).select(
    "title", "deaths", "potential_death", "false_alarm"
).fill_nan(0).fill_null(0)

dbscan = DBSCAN()

res = dbscan.fit_predict(proc_df[["deaths", "potential_death", "false_alarm"]])
proc_df = proc_df.with_columns(clustered=pl.lit(dbscan.labels_))

CPU times: user 1 μs, sys: 0 ns, total: 1 μs
Wall time: 3.81 μs


In [7]:
for v in proc_df["clustered"].unique():
    print(proc_df.filter(proc_df["clustered"] == v))
    print("\n\n")

shape: (13, 5)
┌───────────────────────────┬────────┬─────────────────┬─────────────┬───────────┐
│ title                     ┆ deaths ┆ potential_death ┆ false_alarm ┆ clustered │
│ ---                       ┆ ---    ┆ ---             ┆ ---         ┆ ---       │
│ str                       ┆ f64    ┆ f64             ┆ f64         ┆ i64       │
╞═══════════════════════════╪════════╪═════════════════╪═════════════╪═══════════╡
│ #1                        ┆ 9.0    ┆ 1.0             ┆ 0.0         ┆ -1        │
│ Multiple murders          ┆ 8.0    ┆ 1.0             ┆ 0.0         ┆ -1        │
│ Multiple shootings        ┆ 8.0    ┆ 1.0             ┆ 0.0         ┆ -1        │
│ Interstate pile-up        ┆ 10.0   ┆ 1.0             ┆ 0.0         ┆ -1        │
│ School shooting           ┆ 27.0   ┆ 1.0             ┆ 0.0         ┆ -1        │
│ School shooting           ┆ 27.0   ┆ 1.0             ┆ 0.0         ┆ -1        │
│ School shooting           ┆ 27.0   ┆ 1.0             ┆ 0.0         ┆ -

In [8]:
%time
most_fre_incedents = [
    'shooting', 'murder', 'burglary',
    'fire', 'crash', 'murder-suicide', 
    'stabbing', 'robbery', 'invasion',
]
        

def apply_func(row: str):
    for inc in most_fre_incedents:
        if row.find(inc) != -1:
            return inc


proc_df = proc_df.with_columns(category=proc_df["title"].map_elements(
    apply_func, return_dtype=str, skip_nulls=False
)).drop_nulls()

CPU times: user 2 μs, sys: 0 ns, total: 2 μs
Wall time: 5.48 μs


In [9]:
proc_df

title,deaths,potential_death,false_alarm,clustered,category
str,f64,f64,f64,i64,str
"""Maryland fire 911""",5.0,1.0,0.0,2,"""fire"""
"""Courthouse shooting""",2.0,1.0,0.0,1,"""shooting"""
"""Festival crash – DC""",0.0,1.0,0.0,3,"""crash"""
"""Double murder – Ken.""",2.0,1.0,0.0,1,"""murder"""
"""Teen car shooting – Colo.""",1.0,1.0,0.0,0,"""shooting"""
"""Taxi murders – Tex.""",2.0,1.0,0.0,1,"""murder"""
"""Midair plane crash – Calif.""",5.0,1.0,0.0,2,"""crash"""
"""Pre-murder 911 – Calif.""",1.0,1.0,0.0,0,"""murder"""
"""Accidental shooting – Fla.""",1.0,1.0,0.0,0,"""shooting"""


In [10]:
%time
agg_by_cat_df = proc_df.group_by("category").agg(
    pl.len().alias("incedent_num"),
    pl.col("deaths").sum().alias("death_num"),
    pl.col("deaths").mean().alias("mean_death_num"),
    pl.col("false_alarm").mean(),
    pl.col("clustered").alias("clusters"),
    pl.col("clustered").mode().alias("max_cluster"),
)

agg_by_clust_df = proc_df.group_by("clustered").agg(
    pl.len().alias("incedent_num"),
    pl.col("deaths").sum().alias("death_num"),
    pl.col("deaths").mean().alias("mean_death_num"),
    pl.col("false_alarm").mean(),
    pl.col("title").alias("titles"),
    pl.col("category").alias("categories"),
    pl.col("category").mode().alias("max_category")
)

CPU times: user 2 μs, sys: 1e+03 ns, total: 3 μs
Wall time: 5.96 μs


In [11]:
print(agg_by_cat_df)
print(agg_by_clust_df)

shape: (8, 7)
┌──────────┬──────────────┬───────────┬────────────────┬─────────────┬──────────────┬─────────────┐
│ category ┆ incedent_num ┆ death_num ┆ mean_death_num ┆ false_alarm ┆ clusters     ┆ max_cluster │
│ ---      ┆ ---          ┆ ---       ┆ ---            ┆ ---         ┆ ---          ┆ ---         │
│ str      ┆ u32          ┆ f64       ┆ f64            ┆ f64         ┆ list[i64]    ┆ list[i64]   │
╞══════════╪══════════════╪═══════════╪════════════════╪═════════════╪══════════════╪═════════════╡
│ fire     ┆ 20           ┆ 46.0      ┆ 2.3            ┆ 0.0         ┆ [2, 0, … 6]  ┆ [3]         │
│ invasion ┆ 5            ┆ 2.0       ┆ 0.4            ┆ 0.0         ┆ [4, 4, … 0]  ┆ [0, 4]      │
│ stabbing ┆ 4            ┆ 4.0       ┆ 1.0            ┆ 0.0         ┆ [0, 0, … 3]  ┆ [0]         │
│ shooting ┆ 96           ┆ 208.0     ┆ 2.166667       ┆ 0.0         ┆ [1, 0, … -1] ┆ [0]         │
│ robbery  ┆ 11           ┆ 1.0       ┆ 0.090909       ┆ 0.0         ┆ [3, 4, … 3]  ┆ 

In [12]:
%time
agg_by_cat_df.write_ndjson(AGG_CAT)
agg_by_clust_df.write_ndjson(AGG_CLUST)

CPU times: user 2 μs, sys: 1 μs, total: 3 μs
Wall time: 5.72 μs


In [13]:
%time
res = pl.scan_ndjson(AGG_CAT).filter(
    (
        ((pl.col("mean_death_num") > 2) & pl.col("clusters").list.contains(3)) |
        ((pl.col("death_num") < 10) & pl.col("clusters").list.contains(1))
    ) &
    ((pl.col("incedent_num") > 10) | (pl.col("false_alarm") > 0))
).collect().sort("category")

CPU times: user 2 μs, sys: 0 ns, total: 2 μs
Wall time: 5.25 μs


In [14]:
res

category,incedent_num,death_num,mean_death_num,false_alarm,clusters,max_cluster
str,i64,f64,f64,f64,list[i64],list[i64]
"""crash""",23,87.0,3.782609,0.0,"[3, 2, … 1]","[1, 3]"
"""fire""",20,46.0,2.3,0.0,"[2, 0, … 6]",[3]
"""murder""",50,109.0,2.18,0.0,"[1, 1, … 3]",[0]
"""shooting""",96,208.0,2.166667,0.0,"[1, 0, … -1]",[0]


In [15]:
%time
res = pl.scan_ndjson(AGG_CLUST).filter(
    [pl.col("categories").list.contains(category) for category in ["fire", "crash"]],
    pl.col("incedent_num") < 30
).collect()

CPU times: user 2 μs, sys: 1 μs, total: 3 μs
Wall time: 4.77 μs


In [16]:
res

clustered,incedent_num,death_num,mean_death_num,false_alarm,titles,categories,max_category
i64,i64,f64,f64,f64,list[str],list[str],list[str]
2,3,15.0,5.0,0.0,"[""Maryland fire 911"", ""Midair plane crash – Calif."", ""iHop shooting""]","[""fire"", ""crash"", ""shooting""]","[""shooting"", ""fire"", ""crash""]"
10,4,28.0,7.0,0.0,"[""Jet plane crash"", ""Multiple murders"", … ""Fatal fire""]","[""crash"", ""murder"", … ""fire""]","[""fire""]"
6,14,42.0,3.0,0.0,"[""Hostage-murder"", ""Plane crash"", … ""Fatal fire""]","[""murder"", ""crash"", … ""fire""]","[""shooting""]"
-1,10,174.0,17.4,0.0,"[""Multiple murders"", ""Multiple shootings"", … ""Workplace shooting""]","[""murder"", ""shooting"", … ""shooting""]","[""shooting""]"
